**Inclass material for Week 4: SQL Query using `pandas`**

This notebook was made based on main materials `4_SQL_Query.ipynb`

Version: Theia - September 2022

---
**START OF DAY 1**

# SQL Query using `pandas`

**Training Objectives**

- Querying from SQL Databases
- SQL Joins
- SQL Conditional Statements
- Flavors and Common Operators
- End to end data analysis

# Working with SQL Databases

## Database Schema

Database `chinook.db` terdiri dari 11 tabel yang berisi sampel data dari sebuah toko yang menjual media digital:

- Data terkait media seperti pada tabel `tracks`, `albums`, `artists`, `genres`, `media_types`, `playlists`, `playlist_track` merupakan data asli dari library Apple iTunes.
- Informasi mengenai `customers` dan `employees` dibuat menggunakan nama dan alamat fiktif yang dapat ditemukan di Google maps, dan data lain yang diformat dengan baik (telepon, fax, email, dll).
- Informasi penjualan `invoices` dan `invoice_items` dihasilkan secara otomatis menggunakan data acak untuk periode tahun 2009-2013.

Skema berikut sering disebut sebagai **Entity Relationship Diagram (ERD)**, menunjukkan:
1. Entitas (Tabel)
2. Atribut (Kolom beserta tipe datanya)
3. Kardinalitas (Hubungan antar tabel)

![](assets/chinookschema2.png)

Gambar di atas adalah ERD untuk database chinook:

- Satu database terdiri dari beberapa tabel
- Setiap tabel terdiri dari beberapa atribut/kolom
- Terdapat kolom unik yang membedakan satu tabel dengan lainnya, disebut sebagai **Primary Key (PK)**. PK dapat digunakan sebagai kolom penghubung antar tabel saat dilakukan proses join (penggabungan) tabel.

## Database Connection

Terdapat banyak package Python yang menyediakan fungsionalitas agar data analyst dapat bekerja dengan basis data (database). Berikut adalah contohnya:

<br>

<details>
    <summary>✨ Connecting to MySQL</summary>
    
```python
import pymysql
  
conn = pymysql.connect(
    host = HOST_NAME,
    port = PORT_NUMBER,
    user = USER_NAME,
    password = PASSWORD,
    db = DATABASE_NAME)
```
</details>

<br>

<details>
    <summary>✨ Connecting to Oracle</summary>
    
```python
import cx_Oracle
  
# data source name from tnsnames.ora file
dsn_tns = cx_Oracle.makedsn(
    HOST_NAME,
    PORT_NUMBER
    service_name = SERVICE_NAME)

# connection
conn = cx_Oracle.connect(
    user = USER_NAME,
    password = PASSWORD,
    dsn = dsn_tns)
```
</details>

<br>

<details>
    <summary>✨ Connecting to PostgreSQL</summary>
    
```python
import psycopg2

conn = psycopg2.connect(
    host = HOST_NAME,
    port = PORT_NUMBER,
    user = USER_NAME,
    password = PASSWORD,
    database = DATABASE_NAME)
```
</details>

<br>

<details>
    <summary>✨ Connecting to Microsoft SQL Server</summary>
    
```python
import pyodbc 
conn = pyodbc.connect(
    'Driver={ODBC Driver 17 for SQL Server};'
    'Server=host;'
    'PORT=1433;'
    'UID=user;'
    'PWD=password;'
    'Database=database;')
```
</details>

Kemudian untuk membaca data kita menggunakan `pd.read_sql_query()` dan menyertakan connection yang telah dibuat:

```python
sales = pd.read_sql_query("SELECT * FROM sales", conn)
```

Saat melempar object `conn`, `pandas` menggunakan [SQLAlchemy](https://www.sqlalchemy.org/) sehingga setiap database dapat bekerja. Tenang saja, hal ini bukan sesuatu yang perlu Anda khawatirkan pada pembelajaran ini. Sebagai tahap awal, mari kita coba bagaimana mengkoneksikan Jupyter Notebook dengan database SQLite (menggunakan package `sqlite3`) yang disebut sebagai **connection**:

In [1]:
import sqlite3 # default package: Database API (DBAPI) / package
import pandas as pd

In [ ]:
conn = 

conn

## `SELECT` Statements

`SELECT` digunakan untuk memilih **kolom** dari sebuah **tabel**.

Syntax `SELECT`:

```sql
SELECT <NAMA_KOLOM>
FROM <NAMA_TABLE>
```

❓ Misal, hanya ingin mengambil kolom `AlbumId` dan `Title` dari tabel `albums`

❗️ Gunakan command `SELECT *` untuk mengambil **semua kolom** dari tabel `albums`:

💡 **Note**: Berbeda dari Python, perintah yang ada di SQL bersifat case **insensitive**. Query di atas dapat juga dituliskan sebagai `select * from albums`. Namun, demi kemudahan pembacaan, statement biasa dituliskan dengan huruf besar (dalam hal ini `SELECT` dan `FROM`).

### ❓ Knowledge Check

Kali ini, coba ambil semua kolom pada tabel `artists` kemudian simpan ke dalam sebuah object dataframe bernama `artist`.

**Pertanyaan:** Berapa baris yang ada pada dataframe `artist`?

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>
    
```python
___ = pd.read_sql_query("SELECT ___ FROM ___", conn)
```
</details>

In [24]:
# your code here


🔎 Kita akan sering menggunakan parameter berikut dalam method `pd.read_sql_query()`:

- `sql`: SQL query dalam bentuk string
- `con`: SQL connection
- `index_col`: nama atau index kolom yang ingin dijadikan index (seperti pada `pd.read_csv()`)
- `parse_dates`: nama kolom yang ingin dikonversi menjadi tipe data `datetime64` (seperti pada `pd.read_csv()`)

Silahkan kunjungi [official documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) untuk detail lebih lanjut.

### LIMIT

Statement `LIMIT` digunakan untuk mengambil beberapa baris pertama pada data.

❓ Contoh: Ambil **5 baris pertama** dari tabel `artists` lalu jadikan kolom `ArtistId` sebuah index dengan parameter `index_col`.

statement limit sama seperti method head di pandas dataframe

❓**Problem:**

> Bagaimana jika ingin menggabungkan data dengan informasi yang lengkap

## SQL Joins

Statement `JOIN` digunakan untuk **menggabungkan dua tabel** menjadi satu tabel melalui **kolom penghubung** yang sama. Operasi `JOIN` dapat dilakukan lebih dari satu kali dalam satu SQL query. Berikut adalah diagram ilustrasi beberapa macam operasi SQL `JOIN`:

![](assets/sqljoins.png)
Credit: Data & Object Factory, LLC

- `LEFT JOIN` paling sering digunakan pada sebagian besar skenario bisnis. `LEFT JOIN` mengembalikan semua baris pada tabel kiri **terlepas** dari apakah ada baris yang cocok pada tabel kanan.
- `INNER JOIN` adalah tipe join yang sangat intuitif dan mudah dipahami. Query ini mengembalikan semua baris di tabel kiri yang cocok dengan tabel kanan.

**Note**:

- `RIGHT JOIN` hampir tidak pernah digunakan karena ekuivalen dengan `LEFT JOIN` hanya mengganti peletakkan tabel kiri dan kanannya saja.
- `FULL OUTER JOIN` sangat jarang digunakan. Selain alasan komputasi, tipe join ini mengembalikan semua baris dari kedua tabel terlepas dari apakah ada kecocokan atau tidak, sehingga menghasilkan DataFrame dengan banyak missing value.

Ilustrasi terkait ke-empat tipe join dapat dilihat pada [Visual JOIN](https://docs.google.com/presentation/d/1weIVJiFtIKjPBlR_YW0R88HDN9nsbhPnOIe4UVVz_8M/edit#slide=id.gfee93a1fe8_0_224).

Syntax `JOIN`:

```sql
SELECT <COLUMNS>
FROM <LEFT_TABLE>
[LEFT|INNER] JOIN <RIGHT_TABLE>
ON <LEFT_TABLE>.key = <RIGHT_TABLE>.key
```

Perhatikan kembali skema database `chinook.db`:

1. Tabel `albums`: 
    - `AlbumId`
    - `Title`
    - `ArtistId`


2. Tabel `artists`:
    - `ArtistId`
    - `Name` 
    
**Kasus:** Kita ingin mendapatkan DataFrame yang menampilkan semua baris dengan kolom `AlbumId`, `Title`, dan `Name`. Perhatikan bahwa kolom `Name` terdapat pada tabel `artists`, sedangkan kolom `AlbumId` dan `Title` terdapat pada tabel `albums`.

- Tabel kiri: __
- Tabel kanan: __

In [2]:
# code here


Perhatikan SQL query di atas:
- Tabel `albums` menjadi tabel kiri, sedangkan tabel `artists` menjadi tabel kanan.
- Kedua tabel digabungkan menggunakan operasi `LEFT JOIN` melalui key `ArtistId`.
- Penggunaan karakter backslash (`\`) ditujukan agar ke-empat baris SQL query dibaca sebagai satu kesatuan baris. Namun penggunaan backslash kurang efisien dan memiliki resiko error yang lebih tinggi. **Sebagai alternatif, Anda dapat mengapit SQL query dengan `"""` (petik tiga kali)**

In [4]:
# re-create tabel di atas menggunakan petik tiga kali 


Terkadang kita memiliki nama kolom ataupun tabel yang panjang dan redundan untuk diketik.

Penggunaan statement `AS` dapat digunakan untuk melakukan **aliasing** nama tabel dan nama kolom.

In [5]:
# re-create tabel di atas menggunakan aliasing


⚠️ Hati-hati! Saat melakukan `LEFT JOIN`, peletakkan tabel kiri dan kanan sangat berpengaruh terhadap hasil penggabungan.

Sebagai perbanding, sekarang coba kita tukar tabel `artists` menjadi tabel kiri, sedangkan `albums` menjadi tabel kanan.

Object `x` di bawah menampilkan semua data `artists` walaupun artist tersebut tidak ada di tabel `albums`.

In [ ]:
x = pd.read_sql_query(
    """
    SELECT AlbumId, Title, Name
    FROM artists
    LEFT JOIN albums
    ON artists.ArtistId = albums.ArtistId
    
    """, conn)

# cek baris dengan missing value
x[x.isna().any(axis=1)]

# 📝 Summary Day 1


- **Database dan SQL**

    + **Terminologi**

        - **Database** adalah kumpulan data yang disimpan pada sistem komputer yang terorganisir dan sistematis
        - **Structured Query Language (SQL)** : Bahasa pemrograman untuk database relational
        - **Database relational** : database yang di dalamnya terdapat tabel-tabel yang saling berhubungan atau memiliki relasi
        - **Query** : suatu perintah pada SQL untuk menjalankan/melakukan proses tertentu
        - **Entity Relationship Diagram** : Skema diagram atau blueprint yang menjelaskan struktur dari sebuah database
        - Komponen ERD:
            + Entitas/tabel
            + Atribut tabel(berupa kolom dan tipe datanya)
            + Kardinalitas/hubungan tabel
    
    
- **Working with SQL and Pandas**

Dalam integrasi data dari database utamanya untuk melakukan fetching data, setidaknya dilakukan 2 proses sebagai berikut:

1. 
2. 

- **Statement in SQL**

1. `SELECT` : ___
2. `LIMIT` : ___
3. `AS` : ___
4. `JOIN` : ___
    - Jenis `JOIN`:
        + `INNER JOIN` : 
        + `LEFT JOIN` : 
        + `RIGHT JOIN` : 
        + `OUTER JOIN` : 

Pilihan:
- A. mengembalikan seluruh baris data yang ada pada tabel kiri dan hanya sebagian data yang match pada tabel sebelah kanan
- B. mengembalikan baris data yang match baik di tabel kiri dan tabel kanan (key id tabel A = key id tabel B)
- C. mengembalikan seluruh baris data yang ada pada tabel kanan dan hanya sebagian data yang match pada tabel sebelah kiri. Sangat jarang dipakai untuk proses JOIN tabel
- D. mengembalikan seluruh data yang ada pada tabel kanan dan kiri baik yang match ataupun tidak. Kurang direkomendasikan karena komputasi akan semakin besar dan menghasilkan dataframe yang banyak missing valuesnya

---

`SELECT * _____ albums`

- Query apa yang cocok untuk mengisi _____ diatas? 

- Apa tujuan dari tanda * pada query diatas? 



### ❓ Knowledge Check

Create a `DataFrame` named `tracks`, containing all columns from the `tracks` table; Additionally, it should also contains:

- The `Title` column from the `albums` table
- The `Name` column from the `artists` table
- The `Name` column from the `genres` table

> **Hint 1**: In your `SELECT` statement, you can use `SELECT tracks.* FROM TRACKS` to select all columns from the `TRACKS` table
> 
> **Hint 2**: When we write `SELECT tracks.Name AS tracksName`, we are renaming the output column from `Name` to `tracksName` using a technique called column aliasing. You may optionally consider doing this for columns that share the same name across different tables 

- Set the `TrackId` column to be the index.
- Verify: the resulting `DataFrame` should has 3503 rows and 11 columns. 

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>

> **Hint 1**: Menentukan tabel utama terlebih dahulu, kemudian menentukan tabel yang berelasi dengan tabel utama.
    
> **Hint 2**: Pada kasus ini, Anda wajib menggunakan syntax `<nama_tabel>.<nama_kolom>` pada statement `SELECT`. Hal ini karena terdapat nama kolom yang sama dari tabel yang berbeda. Jangan lupa untuk menggunakan aliasing `AS` agar nama kolom akhir tidak ambigu.
    
> **Hint 3**: Anda dapat menggunakan `SELECT tracks.*` untuk mengambil semua kolom pada tabel `tracks`

> **Hint 4**: Anda dapat melanjutkan hasil operasi `LEFT JOIN` dengan `LEFT JOIN` berikutnya. Sebagai referensi, berikut adalah struktur kodenya:
    
```python
___ = pd.read_sql_query(
    """
    SELECT ___
    FROM ___
    LEFT JOIN ___
    ON ___
    LEFT JOIN ___
    ON ___
    LEFT JOIN ___
    ON ___
    """, conn, index_col=___)
```
</details>

In [ ]:
# your code here
tracks = 

tracks.head()

Perform EDA on `tracks` to answer the following question:

1. Use `tail()` to inspect the last 5 rows of data. Which genre is present in the last 5 rows of our `tracks` DataFrame (Check all that apply)?
    - [ ] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

In [7]:
# your code here


2. Apply `pd.crosstab(..., columns='count')`, `.value_counts()`, or any other techniques you've learned to compute the frequency table of Genres in your DataFrame. Which is among the top 3 most represented genres in the `tracks` DataFrame?
    - [ ] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

In [6]:
# your code here


3. Use `groupby()` on Artist Name and compute the `mean()` on the `UnitPrice` of each tracks. You will realize that most artists price their tracks at 0.99 (`mean`) but there are several artists where the `mean()` is 1.99. Which of the Artist has a mean of 0.99 `UnitPrice`:
    - [ ] The Office
    - [ ] Aquaman
    - [ ] Pearl Jam
    - [ ] Lost

In [5]:
# your code here
